# Shapash Overview

<b>With this tutorial you:</b><br />
Understand how shapash works with a simple use case<br />
Launch the WebApp save these results

Contents:
- Build a Binary Classifier
- Compile Shapash SmartExplainer
- Launch Shapash WebApp
- Export synt with to_pandas function
- Save Shapash object in pickle file

Data from Kaggle [Titanic](https://www.kaggle.com/c/titanic)

In [86]:
import pandas as pd
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

import shapash

## Building Supervized Model 

In [87]:
pathpartage='/Users/11258k/Documents/Projets/Diaphane'

import sys
sys.path.append(pathpartage)

In [88]:
from shapash.data.data_loader import data_loading

In [99]:
house_df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2-Story 1946 & Newer,Residential Low Density,8450,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,Normal,Normal,Single-family Detached,Two story,7,5,2003,2003,Gable,Standard (Composite) Shingle,Vinyl Siding,Vinyl Siding,Brick Face,196.0,Good,Average/Typical,Poured Contrete,Good (90-99 inches),Typical - slight dampness allowed,No Exposure/No Basement,Good Living Quarters,706,Unfinished/No Basement,0,150,856,Gas forced warm air furnace,Excellent,Yes,Standard Circuit Breakers & Romex,856,854,0,1710,1,0,2,1,3,1,Good,8,Typical Functionality,0,Attached to home,2003.0,Rough Finished,548,Typical/Average,Typical/Average,Paved,0,61,0,0,0,0,0,2,2008,Warranty Deed - Conventional,Normal Sale,208500
2,1-Story 1946 & Newer All Styles,Residential Low Density,9600,Paved,Regular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Frontage on 2 sides of property,Gentle slope,Veenker,Adjacent to feeder street,Normal,Single-family Detached,One story,6,8,1976,1976,Gable,Standard (Composite) Shingle,Metal Siding,Metal Siding,None,0.0,Average/Typical,Average/Typical,Cinder Block,Good (90-99 inches),Typical - slight dampness allowed,Good Exposure,Average Living Quarters,978,Unfinished/No Basement,0,284,1262,Gas forced warm air furnace,Excellent,Yes,Standard Circuit Breakers & Romex,1262,0,0,1262,0,1,2,0,3,1,Typical/Average,6,Typical Functionality,1,Attached to home,1976.0,Rough Finished,460,Typical/Average,Typical/Average,Paved,298,0,0,0,0,0,0,5,2007,Warranty Deed - Conventional,Normal Sale,181500
3,2-Story 1946 & Newer,Residential Low Density,11250,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Inside lot,Gentle slope,College Creek,Normal,Normal,Single-family Detached,Two story,7,5,2001,2002,Gable,Standard (Composite) Shingle,Vinyl Siding,Vinyl Siding,Brick Face,162.0,Good,Average/Typical,Poured Contrete,Good (90-99 inches),Typical - slight dampness allowed,Mimimum Exposure,Good Living Quarters,486,Unfinished/No Basement,0,434,920,Gas forced warm air furnace,Excellent,Yes,Standard Circuit Breakers & Romex,920,866,0,1786,1,0,2,1,3,1,Good,6,Typical Functionality,1,Attached to home,2001.0,Rough Finished,608,Typical/Average,Typical/Average,Paved,0,42,0,0,0,0,0,9,2008,Warranty Deed - Conventional,Normal Sale,223500
4,2-Story 1945 & Older,Residential Low Density,9550,Paved,Slightly irregular,Near Flat/Level,"All public Utilities (E,G,W,& S)",Corner lot,Gentle slope,Crawford,Normal,Normal,Single-family Detached,Two story,7,5,1915,1970,Gable,Standard (Composite) Shingle,Wood Siding,Wood Shingles,None,0.0,Average/Typical,Average/Typical,Brick & Tile,Typical (80-89 inches),Good,No Exposure/No Basement,Average Living Quarters,216,Unfinished/No Basement,0,540,756,Gas forced warm air furnace,Good,Yes,Standard Circuit Breakers & Romex,961,756,0,1717,1,0,1,0,3,1,Good,7,Typical Functionality,1,Detached from home,1998.0,Unfinished/No Garage,642,Typical/Average,Typical/Average,Paved,0,35,272,0,0,0,0,2,2006,Warranty Deed - Conventional,Abnormal Sale,140000
5,2-Story 1946 & Newer,Residential Low Density,14260,Paved,Slightly irregular,Near F

In [89]:
house_df, label_dict = data_loading('house_prices')
y_df=house_df['SalePrice'].to_frame()
X_df=house_df[house_df.columns.difference(['SalePrice'])]

#### Encoding Categorical Features 

In [90]:
from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df=encoder.transform(X_df)

#### Train / Test Split

In [91]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=1)

#### Model Fitting

In [92]:
regressor = LGBMRegressor(n_estimators=200).fit(Xtrain,ytrain)

In [93]:
y_pred = pd.DataFrame(regressor.predict(Xtest),columns=['pred'],index=Xtest.index)

## Understanding my model with shapash

#### Declare and Compile SmartExplainer 

In [94]:
from shapash.explainer.smart_explainer import SmartExplainer

In [95]:
xpl = SmartExplainer(features_dict=label_dict) # optional parameter, specifies label for features name 

In [96]:
xpl.compile(
    x=Xtest,
    model=regressor,
    preprocessing=encoder, # Optional: compile step can use inverse_transform method
    y_pred=y_pred # Optional
)

Backend: Shap TreeExplainer


In [83]:
xpl.run_app()

In [97]:
xpl.plot.features_importance()

In [103]:
xpl.plot.contribution_plot("BldgType")

In [7]:
titan_df, titan_dict = data_loading()

In [104]:
xpl.plot.local_plot(row_num=1)

#### Export local explaination in DataFrame

In [106]:
summary_df= xpl.to_pandas(
    max_contrib=3, # Number Max of features to show in summary
    threshold=5000,
)

In [107]:
summary_df.head()

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
259,209141.256921,Ground living area square feet,1792,13710.4,Overall material and finish of the house,7,12776.3,Total square feet of basement area,963,-5103.03
268,178734.474531,Ground living area square feet,2192,29747,Overall material and finish of the house,5,-26151.3,Overall condition of the house,8,9190.84
289,113950.844570,Overall material and finish of the house,5,-24730,Ground living area square feet,900,-16342.6,Total square feet of basement area,882,-5922.64
650,74957.162142,Overall material and finish of the house,4,-33927.7,Ground living area square feet,630,-23234.4,Total square feet of basement area,630,-11687.9
1234,135305.243500,Overall material and finish of the house,5,-25445.7,Ground living area square feet,1188,-11476.6,Condition of sale,Abnormal Sale,-5071.82


#### Save SmartExplainer in Pickle File

You can save the SmartExplainer Object in a pickle file to make new plots later or launch the WebApp again

In [2]:
xpl.save('./xpl.pkl')